<a href="https://colab.research.google.com/github/ss8319/Colab-Notebooks/blob/main/MT5_en_swh_acw_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# Confirm GPU availability
!nvidia-smi

Tue Aug 23 07:11:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 13.3 MB/s 
     |████████████████████████████████| 6.6 MB 42.3 MB/s 
     |████████████████████████████████| 365 kB 47.0 MB/s 
     |████████████████████████████████| 9.1 MB 68.4 MB/s 
     |████████████████████████████████| 1.8 MB 67.0 MB/s 
     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 4.7 MB 58.0 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 181 kB 71.0 MB/s 
     |████████████████████████████████| 157 kB 74.8 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 157 kB 74.6 MB/s 
     |████████████████████████████████| 157 kB 77.1 MB/s 
     |████████████████████████████████| 157 kB 81.1 MB/s 
     |████████████████████████████████| 157 kB 81.2 MB/s 
     |█████████████████████████████

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# %env WANDB_PROJECT= MT5-small-mgq-Swahili-Bible-Translation-runs

In [ ]:
#
# Install SentencePiece 
#

!pip install sentencepiece
!pip install python-dotenv
!pip install sacrebleu
!pip install sklearn

import os
import pandas as pd
from sklearn.model_selection import train_test_split
import sacrebleu

# Tell the SIL NLP tools where to find the Gutenberg resources
os.environ['SIL_NLP_DATA_PATH'] = "/content/drive/Shareddrives/Gutenberg"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Split dataset into train/val/test

In [ ]:
def prepare_translation_datasets(data_path, src_lang, src, trg_lang, trg, test_size):
    src_text = []
    trg_text = []
    with open(os.path.join(data_path, src), "r", encoding="utf-8") as s, \
         open(os.path.join(data_path, trg), "r", encoding="utf-8") as t:
        for src_line, trg_line in zip(s, t):
            src_line = src_line.strip()
            trg_line = trg_line.strip()
            if len(src_line) == 0 or len(trg_line) == 0: #if either the src or text line is an empty line move back to top of for loop(next line)
                continue
            src_text.append(src_line)
            trg_text.append(trg_line)

    X_train, X_test, y_train, y_test = train_test_split(src_text, trg_text, test_size=test_size,random_state=111)


    data = []
    for src, trg in zip(X_train, y_train):
        data.append([f"translate {src_lang} to {trg_lang}", src, trg])
        # data.append([f"translate {trg_lang} to {src_lang}", trg, src])

    train_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])


    data = []
    for src, trg in zip(X_test, y_test):
        data.append([f"translate {src_lang} to {trg_lang}", src, trg])
        # data.append([f"translate {trg_lang} to {src_lang}", trg, src])

    eval_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])

    return train_df, eval_df


In [ ]:
train_df, eval_df = prepare_translation_datasets("/content/gdrive/Shareddrives/Gutenberg/MT/scripture", "english", "en-NIV11R.txt", "swahili", "swh-NEN.txt", 3000)

# English/Swahili Fine-Tuning

In [ ]:
#load train/test/val.txt into a pandas.DataFrame 
import csv

def load_train(data_path, src_lang, src_filename, trg_lang, trg_filename): 
    src_text = []
    trg_text = []
    data = []
    with open(os.path.join(data_path, src_filename), "r", encoding="utf-8") as s, \
         open(os.path.join(data_path, trg_filename), "r", encoding="utf-8") as t:
        for src_line, trg_line in zip(s, t):
           data.append([f"translate {src_lang} to {trg_lang}", src_line, trg_line])
          # data.append([f"translate {trg_lang} to {src_lang}", trg, src])
    train_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])  
    return train_df
train_df=load_train("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mt5-en-swh_MultiCCAligned","english","en-MultiCCAligned_en_swh.clean.all.txt","swahili","swh-MultiCCAligned_en_swh.clean.all.txt")


def load_eval(data_path, src_lang, src_filename, trg_lang, trg_filename): 
    src_text = []
    trg_text = []
    data = []
    with open(os.path.join(data_path, src_filename), "r", encoding="utf-8") as s, \
         open(os.path.join(data_path, trg_filename), "r", encoding="utf-8") as t:
        for src_line, trg_line in zip(s, t):
           data.append([f"translate {src_lang} to {trg_lang}", src_line, trg_line])
          # data.append([f"translate {trg_lang} to {src_lang}", trg, src])
    eval_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])  
    return eval_df
eval_df=load_eval("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mt5-en-swh_MultiCCAligned","english","en-MultiCCAligned_en_swh.clean.val.txt","swahili","swh-MultiCCAligned_en_swh.clean.val.txt")


def load_test(data_path, src_lang, src_filename, trg_lang, trg_filename): 
    src_text = []
    trg_text = []
    data = []
    with open(os.path.join(data_path, src_filename), "r", encoding="utf-8") as s, \
         open(os.path.join(data_path, trg_filename), "r", encoding="utf-8") as t:
        for src_line, trg_line in zip(s, t):
           data.append([f"translate {src_lang} to {trg_lang}", src_line, trg_line])
          # data.append([f"translate {trg_lang} to {src_lang}", trg, src])
    test_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])  
    return test_df
test_df=load_test("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mt5-en-swh_MultiCCAligned","english","en-MultiCCAligned_en_swh.clean.test.txt","swahili","swh-MultiCCAligned_en_swh.clean.test.txt")

In [ ]:
train_df

,prefix,input_text,target_text
0,translate english to swahili,CharityiBonus is a unique community crowdfundi...,CharityiBonus ni jukwaa la kipekee la ukuzaji ...
1,translate english to swahili,20th Century Fox has accused Kim Dotcom of bre...,20th Century Fox ina watuhumiwa Kim Dotcom ya ...
2,translate english to swahili,Dj Maphorisa feat. Wizkid & Dj Buckz – Soweto ...,Diamond Platnumz Ft Flavour – Nana\n
3,translate english to swahili,Just in case you have no idea who the heck EIG...,Tu kama huna wazo ambao EIG ya hekta ni - ni p...
4,translate english to swahili,Aluminium Heat Sink Extrus...\n,Aluminium Joto kuzama Extrus ...\n
...,...,...,...
724955,translate english to swahili,2. Size: 915X1220mm 1220x2440mm 700x1000mm 915...,2. ukubwa: 915X1220mm 1220x2440mm 700x1000mm 9...
724956,translate english to swahili,"Honestly, there’s not much not to like about S...","Kwa kweli, hawana mengi ya kupenda kuhusu Site..."
724957,translate english to swahili,Imam Khomeini International Airport Tehran 39 ...,Ghale Morghi Airport 7 km / 4.1 mi\n
724958,translate english to swahili,IT & Technology (97)\n,IT & Teknolojia (125)\n


In [ ]:
eval_df

,prefix,input_text,target_text
0,translate english to swahili,5. Statement of Purpose.\n,5. Taarifa ya Kusudi.\n
1,translate english to swahili,Loopy Lotto – Land Line Casino\n,Mayan Moola – Amana ya Ardhi Line Casino\n
2,translate english to swahili,Why Buy China Phones? They are cheap and very ...,Why Buy China Phones? Wao ni nafuu na bei nafu...
3,translate english to swahili,Donna shares her TTC journey\n,Donna anashiriki safari yake ya TTC\n
4,translate english to swahili,Island Cabinets Water Sink Kitchen Island Cabi...,Makabati ya Kisiwa Kuzama maji Makabati ya Kis...
...,...,...,...
2995,translate english to swahili,24. The large RIP tattoo on the male arm\n,24. RIP kubwa ya tattoo juu ya mkono wa kiume\n
2996,translate english to swahili,The “smallest bit” of self-righteousness must ...,Hata “ile kiasi ndogo ya haki ya kibinafsi laz...
2997,translate english to swahili,"Wednesday, 20 March 2019 08:06\n","Jumatano, 20 Machi 2019 08:06\n"
2998,translate english to swahili,To adapt to the partner said to have intercour...,Kukabiliana na mpenzi alisema kuwa ngono na ki...


In [ ]:
test_df

,prefix,input_text,target_text
0,translate english to swahili,YKG Gallery _ YKG Summer Holidays\n,© 2019 YKG / Yutaka Kikutake Gallery\n
1,translate english to swahili,Sceneries are not automatically enabled with P...,Maonyesho hayajawezeshwa moja kwa moja na Prep...
2,translate english to swahili,Copyright © 2018. All Rights Reserved.\n,Hakinakili © 2018. Haki zote zimehifadhiwa.\n
3,translate english to swahili,"For when ELOHIYM made promise to Abraham, beca...","Mungu alipompa Abrahamu ahadi, aliapa kwa jina..."
4,translate english to swahili,"And the angel said unto me, Wherefore didst th...","Lakini malaika akaniambia, ""Kwa nini unashanga..."
...,...,...,...
2995,translate english to swahili,There are 2 different packages o…\n,Kuna vifurushi tofauti vya 2 o…\n
2996,translate english to swahili,29 And Jacob sod pottage: and Esau came from t...,29 Yakobo akapika chakula cha dengu. Esau akaj...
2997,translate english to swahili,Favicons: Icon made by Freepik from www.flatic...,NGB Ningbo Lishe International Airport 5.70 mi...
2998,translate english to swahili,"Writing the course, and possibly recording it,...","Kuandika kozi hiyo, na ikiwezekana kurekodi, n..."


In [ ]:
from simpletransformers.t5 import T5Model, T5Args
model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 20
model_args.eval_batch_size = 20
model_args.num_train_epochs = 1
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 English-Swahili Translation"

model = T5Model("mt5", "google/mt5-base", args=model_args)

In [ ]:
# Train the model
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/724960 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Running Epoch 0 of 1:   0%|          | 0/36248 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
!pip install sacrebleu
import sacrebleu
import logging
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 512
model_args.length_penalty = 1
model_args.num_beams = 10

model = T5Model("mt5", "outputs", args=model_args)

In [ ]:
#create test set
swahili_truth = [test_df.loc[test_df["prefix"] == "translate english to swahili"]["target_text"].tolist()]
to_swahili = test_df.loc[eval_df["prefix"] == "translate english to swahili"]["input_text"].tolist()

#for bidirectional use
# limbu_truth = [eval_df.loc[eval_df["prefix"] == "translate nepali to limbu"]["target_text"].tolist()]
# to_limbu = eval_df.loc[eval_df["prefix"] == "translate nepali to limbu"]["input_text"].tolist()
# Predict
swahili_preds = model.predict(to_swahili)
bleu = sacrebleu.corpus_bleu(swahili_preds, swahili_truth)
print("--------------------------")
print("English to Swahili: ", bleu.score)


In [ ]:
print(f"{bleu.score}/{bleu.precisions[0]}/{bleu.precisions[1]}/{bleu.precisions[2]}/{bleu.precisions[3]}/{bleu.bp}")

In [ ]:
#save predictions into .txt file
# add '\n' after each item of a list
swahili_preds = ["{}\n".format(i) for i in swahili_preds]
# swahili_truth = ["{}".format(i) for i in swahili_truth]


with open('/content/src_pred.txt','w',encoding='utf-8') as f:
  f.writelines(swahili_preds)

# with open('/content/trg_truth.txt','w',encoding='utf-8') as f:
#   f.writelines(swahili_truth)

# English/Swahili or XX/Swahili Bible Fine-Tuning

In [ ]:
#load train/test/val.txt into a pandas.DataFrame 
import csv

def load_train(data_path, src_lang, src_filename, trg_lang, trg_filename): 
    src_text = []
    trg_text = []
    data = []
    with open(os.path.join(data_path, src_filename), "r", encoding="utf-8") as s, \
         open(os.path.join(data_path, trg_filename), "r", encoding="utf-8") as t:
        for src_line, trg_line in zip(s, t):
           data.append([f"translate {src_lang} to {trg_lang}", src_line, trg_line])
          # data.append([f"translate {trg_lang} to {src_lang}", trg, src])
    train_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])  
    return train_df
train_df=load_train("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mt5-mgq-swh3.0","swahili","mgq-MALILA_train.txt","swahili","swh-MALILABT_train.txt")


def load_eval(data_path, src_lang, src_filename, trg_lang, trg_filename): 
    src_text = []
    trg_text = []
    data = []
    with open(os.path.join(data_path, src_filename), "r", encoding="utf-8") as s, \
         open(os.path.join(data_path, trg_filename), "r", encoding="utf-8") as t:
        for src_line, trg_line in zip(s, t):
           data.append([f"translate {src_lang} to {trg_lang}", src_line, trg_line])
          # data.append([f"translate {trg_lang} to {src_lang}", trg, src])
    eval_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])  
    return eval_df
eval_df=load_eval("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mt5-mgq-swh3.0","swahili","mgq-MALILA_val.txt","swahili","swh-MALILABT_val.txt")


# def load_test(data_path, src_lang, src_filename, trg_lang, trg_filename): 
#     src_text = []
#     trg_text = []
#     data = []
#     with open(os.path.join(data_path, src_filename), "r", encoding="utf-8") as s, \
#          open(os.path.join(data_path, trg_filename), "r", encoding="utf-8") as t:
#         for src_line, trg_line in zip(s, t):
#            data.append([f"translate {src_lang} to {trg_lang}", src_line, trg_line])
#           # data.append([f"translate {trg_lang} to {src_lang}", trg, src])
#     test_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])  
#     return test_df
# test_df=load_test("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mt5-mgq-swh2.0","swahili","mgq-MALILA_test.txt","swahili","swh-MALILABT_test.txt")

FileNotFoundError: ignored

In [ ]:
train_df

In [ ]:
eval_df

In [ ]:
test_df

In [ ]:
#saving dataFrame into a tsv for storage (not working)
train_df.to_csv("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mT5-mgq-swh2.0/train.tsv", sep="\t")
train_df = pd.read_csv("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mT5-mgq-swh2.0/train.tsv", sep="\t").astype(str)
train_df.to_csv("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mT5-mgq-swh/mgq-MALILA-swh-MALILABT_train.tsv", sep="\t")
eval_df.to_csv("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mT5-mgq-swh/mgq-MALILA-swh-MALILABT_eval.tsv", sep="\t")
train_df = pd.read_csv("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mT5-mgq-swh/mgq-MALILA-swh-MALILABT_train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mT5-mgq-swh/mgq-MALILA-swh-MALILABT_eval.tsv", sep="\t").astype(str)

#train_df["prefix"] = ""
#eval_df["prefix"] = ""

In [ ]:
import logging
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
#Automated HyperParameter Tuning with W&B
import wandb
sweep_config = {
    "method": "random",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [5,10,15,20]},
        "learning_rate": {"min": 7e-5, "max": 2e-4},
        "batch size":{"min": 5, "max": 20}
    },
}

sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")
def train():
    # Initialize a new wandb run
    wandb.init()

    model_args = T5Args()
    model_args.max_seq_length = 96
    model_args.train_batch_size = 10
    model_args.eval_batch_size = 10
    model_args.num_train_epochs = 1
    model_args.evaluate_during_training = True
    model_args.evaluate_during_training_steps = 30000
    model_args.use_multiprocessing = False
    model_args.fp16 = False
    model_args.save_steps = -1
    model_args.save_eval_checkpoints = False
    model_args.no_cache = True
    model_args.reprocess_input_data = True
    model_args.overwrite_output_dir = True
    model_args.preprocess_inputs = False
    model_args.num_return_sequences = 1
    model_args.wandb_project = "MT5 mgq-Swahili Bible Translation sweep"

    model = T5Model("mt5", "google/mt5-large", args=model_args) #mt5-base/large/xl/xxl

    # Train the model
    model.train_model(train_df, eval_data=eval_df)

    # Evaluate the model
    model.eval_model(eval_df)

    # Sync wandb
    wandb.join()


wandb.agent(sweep_id, train)

In [ ]:
model_args = T5Args()
model_args.use_early_stopping = True
model_args.learning_rate=2.5e-5
# model_args.early_stopping_delta = 0.05
# model_args.early_stopping_metric = "eval_loss" #mcc
# model_args.early_stopping_metric_minimize = True
# model_args.early_stopping_patience = 10
# model_args.evaluate_during_training_steps = 10
model_args.max_seq_length = 96
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.num_train_epochs = 10
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5-small-mgq-Swahili-Bible-Translation-runs"

model = T5Model("mt5", "google/mt5-small", args=model_args) #mt5-base/large/xl/xxl

In [ ]:
#CUDA memory management stuff
import torch
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [ ]:
# Train and evaluate the model
import sklearn
model.train_model(train_df, eval_data=eval_df)
#model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

In [ ]:
model_args = T5Args()
model_args.max_length = 512
model_args.length_penalty = 1
model_args.num_beams = 5

model = T5Model("mt5", "outputs", args=model_args)

In [ ]:
#create test set
swahili_truth = [eval_df.loc[eval_df["prefix"] == "translate swahili to swahili"]["target_text"].tolist()]
to_swahili = eval_df.loc[eval_df["prefix"] == "translate swahili to swahili"]["input_text"].tolist()

#for bidirectional use
# limbu_truth = [eval_df.loc[eval_df["prefix"] == "translate nepali to limbu"]["target_text"].tolist()]
# to_limbu = eval_df.loc[eval_df["prefix"] == "translate nepali to limbu"]["input_text"].tolist()
# Predict
swahili_preds = model.predict(to_swahili)
bleu = sacrebleu.corpus_bleu(swahili_preds, swahili_truth)
print("--------------------------")
print("XX to Swahili: ", bleu.score)



# limbu_preds = model.predict(to_limbu)
# nep_lif_bleu = sacrebleu.corpus_bleu(limbu_preds, limbu_truth)
# print("Nepali to Limbu: ", nep_lif_bleu.score)


Generating outputs:   0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/250 [00:00<?, ?it/s]

--------------------------
XX to Swahili:  0.1368930709090095


In [ ]:
swahili_preds


['Kwa sababu Yesu akamwambia akasema, ‘Mtu ambaye alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuwa alikuw

## **Testing**


In [ ]:
model.eval_model(test_df)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/250 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/outputs_cached_128250


Running Evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': nan}


{'eval_loss': nan}

In [ ]:
print(f"{bleu.score}/{bleu.precisions[0]}/{bleu.precisions[1]}/{bleu.precisions[2]}/{bleu.precisions[3]}/{bleu.bp}")

2.0502685090218944/12.873185428649686/3.4756610296178643/1.0331962116138909/0.38224051749485444/1.0


In [ ]:
#save predictions into .txt file
# add '\n' after each item of a list
swahili_preds = ["{}\n".format(i) for i in swahili_preds]
# swahili_truth = ["{}".format(i) for i in swahili_truth]


with open('/content/src_pred.txt','w',encoding='utf-8') as f:
  f.writelines(swahili_preds)

# with open('/content/trg_truth.txt','w',encoding='utf-8') as f:
#   f.writelines(swahili_truth)


In [ ]:
swahili_preds

In [ ]:
swahili_truth


In [ ]:
from google.colab import files
files.download('/content/runs')
files.download('/content/wandb')

#Arabic Inferencing


In [ ]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")
article = "UN Offizier sagt, dass weiter verhandelt werden muss in Syrien."
summary = "Weiter Verhandlung in Syrien."
inputs = tokenizer(article, return_tensors="pt")
with tokenizer.as_target_tokenizer():
    labels = tokenizer(summary, return_tensors="pt")

outputs = model(**inputs, labels=labels["input_ids"])
loss = outputs.loss

In [ ]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")
article = "translate to english: وهما محتارات، وقف فجأة جنبهم رجالين بتايب براقه."
batch = tokenizer.prepare_seq2seq_batch(src_texts=[article], return_tensors="pt")
output_ids = model.generate(input_ids=batch.input_ids, num_return_sequences=1, num_beams=8, length_penalty=0.1)
tokenizer.decode(output_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). C

'<pad> <extra_id_0></s>'

# Arabic Dialect/English Bible Fine-Tuning

In [ ]:
#load train/test/val.txt into a pandas.DataFrame 
import csv

def load_train(data_path, src_lang, src_filename, trg_lang, trg_filename): 
    src_text = []
    trg_text = []
    data = []
    with open(os.path.join(data_path, src_filename), "r", encoding="utf-8") as s, \
         open(os.path.join(data_path, trg_filename), "r", encoding="utf-8") as t:
        for src_line, trg_line in zip(s, t):
           data.append([f"translate {src_lang} to {trg_lang}", src_line, trg_line])
          # data.append([f"translate {trg_lang} to {src_lang}", trg, src])
    train_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])  
    return train_df
train_df=load_train("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mt5-acw-en","arabic","acw-v2-train.txt","english","en-ESVUS16-train.txt")


def load_eval(data_path, src_lang, src_filename, trg_lang, trg_filename): 
    src_text = []
    trg_text = []
    data = []
    with open(os.path.join(data_path, src_filename), "r", encoding="utf-8") as s, \
         open(os.path.join(data_path, trg_filename), "r", encoding="utf-8") as t:
        for src_line, trg_line in zip(s, t):
           data.append([f"translate {src_lang} to {trg_lang}", src_line, trg_line])
          # data.append([f"translate {trg_lang} to {src_lang}", trg, src])
    eval_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])  
    return eval_df
eval_df=load_eval("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mt5-acw-en","arabic","acw-v2-250val.txt","english","en-ESVUS16-250val.txt")


def load_test(data_path, src_lang, src_filename, trg_lang, trg_filename): 
    src_text = []
    trg_text = []
    data = []
    with open(os.path.join(data_path, src_filename), "r", encoding="utf-8") as s, \
         open(os.path.join(data_path, trg_filename), "r", encoding="utf-8") as t:
        for src_line, trg_line in zip(s, t):
           data.append([f"translate {src_lang} to {trg_lang}", src_line, trg_line])
          # data.append([f"translate {trg_lang} to {src_lang}", trg, src])
    test_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])  
    return test_df
test_df=load_test("/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/mt5-acw-en","arabic","acw-v2-250test.txt","english","en-ESVUS16-250test.txt")


In [ ]:
train_df

In [ ]:
eval_df

In [ ]:
test_df

In [ ]:
import logging
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
#Automated HyperParameter Tuning with W&B
import wandb
sweep_config = {
    "method": "random",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [5,10,15,20]},
        "learning_rate": {"min": 7e-5, "max": 2e-4},
        "batch size":{"min": 5, "max": 20}
    },
}

sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")
def train():
    # Initialize a new wandb run
    wandb.init()

    model_args = T5Args()
    model_args.max_seq_length = 96
    model_args.train_batch_size = 10
    model_args.eval_batch_size = 10
    model_args.num_train_epochs = 1
    model_args.evaluate_during_training = True
    model_args.evaluate_during_training_steps = 30000
    model_args.use_multiprocessing = False
    model_args.fp16 = False
    model_args.save_steps = -1
    model_args.save_eval_checkpoints = False
    model_args.no_cache = True
    model_args.reprocess_input_data = True
    model_args.overwrite_output_dir = True
    model_args.preprocess_inputs = False
    model_args.num_return_sequences = 1
    model_args.wandb_project = "MT5 mgq-Swahili Bible Translation sweep"

    model = T5Model("mt5", "google/mt5-large", args=model_args) #mt5-base/large/xl/xxl

    # Train the model
    model.train_model(train_df, eval_data=eval_df)

    # Evaluate the model
    model.eval_model(eval_df)

    # Sync wandb
    wandb.join()


wandb.agent(sweep_id, train)

In [ ]:
model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 5
model_args.eval_batch_size = 5
model_args.num_train_epochs = 5
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 500
model_args.max_steps=5000
model_args.use_multiprocessing = False
model_args.gradient_accumulation_steps=4
model_args.gradient_checkpointing=True
model_args.fp16 = True
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5-base-acw-en"

model = T5Model("mt5", "google/mt5-base", args=model_args) #mt5-base/large/xl/xxl

KeyboardInterrupt: ignored

In [ ]:
#CUDA memory management stuff
import torch
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [ ]:
# Train and evaluate the model
import sklearn
model.train_model(train_df, eval_data=eval_df)
#model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/3264 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Running Epoch 0 of 5:   0%|          | 0/164 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
model_args = T5Args()
model_args.max_length = 512
model_args.length_penalty = 1
model_args.num_beams = 5

model = T5Model("mt5", "outputs", args=model_args)

In [ ]:
#create test set
swahili_truth = [eval_df.loc[eval_df["prefix"] == "translate swahili to swahili"]["target_text"].tolist()]
to_swahili = eval_df.loc[eval_df["prefix"] == "translate swahili to swahili"]["input_text"].tolist()

#for bidirectional use
# limbu_truth = [eval_df.loc[eval_df["prefix"] == "translate nepali to limbu"]["target_text"].tolist()]
# to_limbu = eval_df.loc[eval_df["prefix"] == "translate nepali to limbu"]["input_text"].tolist()
# Predict
swahili_preds = model.predict(to_swahili)
bleu = sacrebleu.corpus_bleu(swahili_preds, swahili_truth)
print("--------------------------")
print("XX to Swahili: ", bleu.score)




Generating outputs:   0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/250 [00:00<?, ?it/s]

--------------------------
XX to Swahili:  0.1368930709090095


In [ ]:
swahili_preds


## **Testing**


In [ ]:
model.eval_model(test_df)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/250 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/outputs_cached_128250


Running Evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': nan}


{'eval_loss': nan}

In [ ]:
print(f"{bleu.score}/{bleu.precisions[0]}/{bleu.precisions[1]}/{bleu.precisions[2]}/{bleu.precisions[3]}/{bleu.bp}")

2.0502685090218944/12.873185428649686/3.4756610296178643/1.0331962116138909/0.38224051749485444/1.0


In [ ]:
#save predictions into .txt file
# add '\n' after each item of a list
swahili_preds = ["{}\n".format(i) for i in swahili_preds]
# swahili_truth = ["{}".format(i) for i in swahili_truth]


with open('/content/src_pred.txt','w',encoding='utf-8') as f:
  f.writelines(swahili_preds)

# with open('/content/trg_truth.txt','w',encoding='utf-8') as f:
#   f.writelines(swahili_truth)


In [ ]:
swahili_preds

In [ ]:
swahili_truth


In [ ]:
from google.colab import files
files.download('/content/runs')
files.download('/content/wandb')